In [51]:
"""
CS-340 Project Two Dashboard
Grazioso Salvare Animal Rescue Data Dashboard

This web dashboard allows Grazioso Salvare staff to interactively filter, view, and visualize animal rescue records stored in MongoDB.
Built with Dash, the dashboard features:
- Interactive filtering by rescue type
- Data table with pagination, sorting, and selection
- Dynamic breed distribution chart
- Interactive geolocation map
- Grazioso Salvare branding and unique identifier

Author: Fnu Samim
Date: 2025-06-22
"""

# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt



# Import the AnimalShelter CRUD class
from animal_shelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# MongoDB connection parameters and class instantiation
# CHANGED: Updated with real credentials and MongoDB config
HOST = 'nv-desktop-services.apporto.com'
PORT = 32575
DB = 'AAC'
COL = 'animals'
shelter = AnimalShelter("aacuser", "SNHU1234")


# Retrieve all animal records as a DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Remove MongoDB's _id column to avoid DataTable compatibility issues
df.drop(columns=['_id'],inplace=True)


#########################
# Dashboard Layout / View
#########################

app = JupyterDash(__name__)

# Load and encode Grazioso Salvare logo
image_filename = 'Grazioso Salvare Logo.png' 
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

# Load and encode a unique identifier image for the dashboard header
car_filename = 'Audi R8 LMS EVO.jpg'  
encoded_car = base64.b64encode(open(car_filename, 'rb').read())

# Define the dashboard layout
app.layout = html.Div([
    # Header section with logos and title
    html.Div([
        # Images container - side by side
        html.Div([
            html.A([
                html.Img(
                    src='data:image/png;base64,{}'.format(encoded_image.decode()),
                    style={
                        'height': '80px',
                        'marginRight': '20px'
                    }
                )
            ], href='https://www.snhu.edu', target='_blank'),
            html.Img(
                src='data:image/jpeg;base64,{}'.format(encoded_car.decode()),
                style={
                    'height': '80px',
                    'marginLeft': '20px'
                }
            )
        ], style={
            'display': 'flex', 
            'justifyContent': 'center', 
            'alignItems': 'center',
            'marginBottom': '20px'
        }),
        html.Center(html.B(html.H1('CS-340 Dashboard - Fnu Samim'))),
    ], style={'textAlign': 'center', 'marginBottom': '20px'}),
    
    html.Hr(),
    
    # Filter section
    html.Div([
        html.Div([
            html.Label('Filter by Rescue Type:', 
                style={'fontWeight': 'bold', 'marginBottom': '10px', 'display': 'block'}),
            dcc.Dropdown(
                id='filter-type',
                options=[
                    {'label': 'All Animals', 'value': 'all'},
                    {'label': 'Water', 'value': 'water'},
                    {'label': 'Mountain/Wilderness', 'value': 'mountain'},
                    {'label': 'Disaster/Individual Tracking', 'value': 'disaster'}
                ],
                value='all',
                style={'width': '300px', 'margin': '0 auto'}
            )
        ], style={'textAlign': 'center'})
    ], style={'margin': '20px 0'}),
    
    html.Hr(),
    
    # Data table section with pagination, filtering, and selection
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),
            filter_action="native",
            sort_action="native",
            page_action="native",
            page_current=0,
            page_size=10,
            row_selectable="single",
            selected_rows=[0],
            style_table={'overflowX': 'auto'},
            style_cell={
                'textAlign': 'left',
                'minWidth': '100px', 
                'width': '150px', 
                'maxWidth': '200px',
                'whiteSpace': 'normal',
                'height': 'auto',
                'padding': '10px'
            },
            style_header={
                'backgroundColor': 'rgb(230, 230, 230)',
                'fontWeight': 'bold'
            },
            style_data_conditional=[
                {
                    'if': {'row_index': 'odd'},
                    'backgroundColor': 'rgb(248, 248, 248)'
                }
            ]
        )
    ], style={'margin': '20px 0'}),
    
    html.Br(),
    html.Hr(),
    
    # Charts section: breed distribution and geolocation
    #This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(
        className='row',
        style={'display': 'flex', 'justifyContent': 'space-around'},
        children=[
            html.Div([
                html.H3('Breed Distribution', style={'textAlign': 'center'}),
                html.Div(id='graph-id')
            ], className='col s12 m6', style={'width': '48%'}),
            
            html.Div([
                html.H3('Geolocation', style={'textAlign': 'center'}),
                html.Div(id='map-id')
            ], className='col s12 m6', style={'width': '48%'})
        ]
    )
])


#############################################
# Interaction Between Components / Controller
#############################################

    
@app.callback(Output('datatable-id', 'data'),
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
    """
    Filter the data table based on rescue type selection.

    Parameters:
        filter_type (str): The rescue type filter selected by the user.
            'all' = All animals (no filter)
            'water' = Water rescue breeds, specific age and sex
            'mountain' = Mountain/Wilderness rescue breeds, specific age and sex
            'disaster' = Disaster/Individual Tracking breeds, specific age and sex

    Returns:
        list of dict: Filtered records to display in the data table.
    """
    if filter_type == 'all':
        # Return all animals
        filtered_df = df
    elif filter_type == 'water':
        # Filter for water rescue animals
        filtered_df = df[
            (df['breed'].isin(['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland'])) &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156) &
            (df['sex_upon_outcome'] == 'Intact Female')
        ]
    elif filter_type == 'mountain':
        # Filter for mountain/wilderness rescue animals
        filtered_df = df[
            (df['breed'].isin(['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog', 'Siberian Husky', 'Rottweiler'])) &
            (df['age_upon_outcome_in_weeks'] >= 26) &
            (df['age_upon_outcome_in_weeks'] <= 156) &
            (df['sex_upon_outcome'] == 'Intact Male')
        ]
    elif filter_type == 'disaster':
        # Filter for disaster/individual tracking animals
        filtered_df = df[
            (df['breed'].isin(['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound', 'Rottweiler'])) &
            (df['age_upon_outcome_in_weeks'] >= 20) &
            (df['age_upon_outcome_in_weeks'] <= 300) &
            (df['sex_upon_outcome'] == 'Intact Male')
        ]
    else:
        filtered_df = df
    
    return filtered_df.to_dict('records')

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    """
    Update the breed distribution pie chart based on filtered data table.

    Parameters:
        viewData (list of dict): The current records displayed in the data table.

    Returns:
        list: Dash Graph component displaying the top 10 breed distribution as a pie chart.
    """
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available for chart")]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Create breed distribution chart
    breed_counts = dff['breed'].value_counts().head(10)  # Top 10 breeds
    
    return [
        dcc.Graph(
            figure=px.pie(
                values=breed_counts.values,
                names=breed_counts.index
            )
        )
    ]

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    """
    Highlight selected columns in the data table.

    Parameters:
        selected_columns (list or None): List of selected column IDs.

    Returns:
        list: Conditional style dictionaries for DataTable styling.
    """
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in (selected_columns or [])]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    """
    Update the geolocation map based on selected row from the data table.

    Parameters:
        viewData (list of dict): The current records displayed in the data table.
        index (list of int): Indices of selected rows (only one is allowed).

    Returns:
        list: Dash Leaflet Map component centered on the selected animal's location.
    """
    if viewData is None or len(viewData) == 0:
        # Default to Austin, TX if no data
        return [
            dl.Map(
                style={'width': '100%', 'height': '500px'},
                center=[30.75, -97.48], 
                zoom=10,
                children=[dl.TileLayer(id="base-layer-id")]
            )
        ]
    
    dff = pd.DataFrame.from_dict(viewData)
    
    # Determine which row to display
    if index is None or len(index) == 0:
        row = 0
    else:
        row = index[0]
    
    # Get location and animal info
    if row < len(dff):
        try:
            lat = float(dff.iloc[row, 13]) if pd.notna(dff.iloc[row, 13]) else 30.75
            lon = float(dff.iloc[row, 14]) if pd.notna(dff.iloc[row, 14]) else -97.48
            breed = str(dff.iloc[row, 4]) if pd.notna(dff.iloc[row, 4]) else "Unknown"
            name = str(dff.iloc[row, 9]) if pd.notna(dff.iloc[row, 9]) else "Unknown"
        except (ValueError, IndexError):
            lat, lon = 30.75, -97.48
            breed, name = "Unknown", "Unknown"
    else:
        lat, lon = 30.75, -97.48
        breed, name = "Unknown", "Unknown"
    
    return [
        dl.Map(
            style={'width': '100%', 'height': '500px'},
            center=[lat, lon], 
            zoom=10, 
            children=[
                dl.TileLayer(id="base-layer-id"),
                dl.Marker(
                    position=[lat, lon],
                    children=[
                        dl.Tooltip(breed),
                        dl.Popup([
                            html.H4("Animal Information"),
                            html.P(f"Name: {name}"),
                            html.P(f"Breed: {breed}")
                        ])
                    ]
                )
            ]
        )
    ]



app.run_server(debug=True)


Dash app running on http://127.0.0.1:18147/
